<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/ChemicalEquilibrium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://reaktoro.org/installation/installation-in-google-colab.html
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh")
!conda config --remove channels defaults
!conda config --add channels conda-forge
!conda install reaktoro -y

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:28
🔁 Restarting kernel...
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
Solving envir

https://reaktoro.org/tutorials/equilibrium/equilibrium-basics.html


In [90]:
from reaktoro import *
import numpy as np
import pandas as pd
import math as math

db = NasaDatabase("nasa-cea")
db = NasaDatabase.withName("nasa-cea")

#print(f"{'Species':<20}{'Formula':<60}{'Molar Mass (kg/mol)':<20}")
#for species in db.species():
#    print(f"{species.name():<20}{species.formula().str():<60}{species.molarMass():<20.6f}")

rxn = db.reaction("CO + 2*H2 = CH3OH")

rprops = rxn.props(25.0, "C", 1.0, "atm")
print(rprops)

#db = SupcrtDatabase("supcrtbl")
#db = SupcrtDatabase.withName("supcrtbl")

#gases = GaseousPhase("N2 CH4 C2H6 O2 CO2 CO H2O H2 NO NO2")
gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")
gases.set(ActivityModelPengRobinson())

solution = CondensedPhase("H2O(l) CH3OH(l)")

system = ChemicalSystem(db, gases,solution)

+------------------------------------------------+-------------+-------------+
| Property                                       |       Value |        Unit |
+------------------------------------------------+-------------+-------------+
| Temperature                                    |    298.1500 |           K |
| Pressure                                       |      1.0132 |         bar |
| Equilibrium Constant (log base 10)             |      4.3880 |           - |
| Delta Standard Gibbs Energy                    | -25046.6852 |       J/mol |
| Delta Standard Enthalpy                        | -90404.2836 |       J/mol |
| Delta Standard Volume                          |  0.0000e+00 |      m3/mol |
| Delta Standard Volume (Temperature Derivative) |  0.0000e+00 |  m3/(mol*K) |
| Delta Standard Volume (Pressure Derivative)    |  0.0000e+00 | m3/(mol*Pa) |
| Delta Standard Isobaric Heat Capacity          |    -42.7743 |   J/(mol*K) |
| Delta Standard Isochoric Heat Capacity         |  

In [2]:
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(1000, "bar")
state.set("CH4", 1.0, "mol")
#state.set("C2H6", 0.04, "mol")
state.set("O2",  1.5, "mol")
#state.set("N2",  0.5, "mol")

print("=== INITIAL STATE ===")
print(state)

=== INITIAL STATE ===
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1273.1500 |    K |
| Pressure        |  1000.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0000e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 3.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.0000e+00 |  mol |
| :: O2           | 1.5000e+00 |  mol |
| :: CO2          | 1.0000e-16 |  mol |
| :: CO           | 1.0000e-16 |  mol |
| :: H2O          | 1.0000e-16 |  mol |
| :: H2           | 1.0000e-16 |  mol |
+-----------------+------------+------+


In [ ]:
solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

In [ ]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

equilibrate(state)

print(state)

In [ ]:
props0 = ChemicalProps(state)

V0 = props0.volume()          # the initial volume of the gases
U0 = props0.internalEnergy()  # the initial internal energy of the gases

print("Initial volume of the gases is", V0, "m3")
print("Initial internal energy of the gases is", U0, "J")

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase("H2O H+ OH- Na+ Cl- HCO3- CO2 CO3-2")
solution.set(ActivityModelPitzer())

system = ChemicalSystem(db, solution)


specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

solver = EquilibriumSolver(specs)

state = ChemicalState(system)
state.temperature(30.0, "celsius")
state.pressure(1.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.4, "mol")

print("INITIAL STATE")
print(state)

conditions = EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "atm")
conditions.pH(1.0)

solver.solve(state, conditions)

print("FINAL STATE")
print(state)

In [ ]:
from reaktoro import *
import numpy as np
import pandas as pd
import math as math

from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("N2 CH4 C2H6 O2 CO2 CO H2O NO NO2")

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(1200, "celsius")
state.pressure(10, "bar")
state.set("CH4", 0.1, "mol")
state.set("C2H6", 0.004, "mol")
state.set("O2",  1.5, "mol")
state.set("N2",  10.5, "mol")

print("=== INITIAL STATE ===")
print(state)

solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

https://en.wikipedia.org/wiki/Flue_gas

https://www.epa.gov/sites/default/files/2020-09/documents/1.4_natural_gas_combustion.pdf

http://cleanboiler.org/workshop/how-is-nox-formed/

https://www.process-heating.com/articles/85910-flame-temperature-what-is-it

https://en.wikipedia.org/wiki/Adiabatic_flame_temperature



# Production of methanol

Pre reforming

H2O/C = 1.5-2

When higher hydrocarbons are present in the natural gas, it is advantageous to
install a pre-reformer upstream the steam reformer in order to avoid carbon
formationC, higher hydrocarbons are
converted into CH4
, CO2
, CO, and H2

In [1]:
from reaktoro import *
import numpy as np
import pandas as pd
import math as math

from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 C2H6 O2 CO H2O H2 CO2 CH3OH")
gases.set(ActivityModelPengRobinson())

aqPhase = CondensedPhase("H2O(l) CH3OH(l)")
#aqPhase.set(ActivityModelPitzer())
#solution = CondensedPhase("H2O(l) CH3OH(l)")

system = ChemicalSystem(db, gases, aqPhase)

state = ChemicalState(system)
state.temperature(460, "celsius")
state.pressure(36, "bar")
state.set("CH4", 1.0, "mol")
state.set("C2H6", 0.05, "mol")
state.set("H2O",  2.0, "mol")

print("=== INITIAL STATE ===")
print(state)

solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== INITIAL STATE ===
+---------------------+------------+------+
| Property            |      Value | Unit |
+---------------------+------------+------+
| Temperature         |   733.1500 |    K |
| Pressure            |    36.0000 |  bar |
| Charge:             | 0.0000e+00 |  mol |
| Element Amount:     |            |      |
| :: H                | 8.3000e+00 |  mol |
| :: C                | 1.1000e+00 |  mol |
| :: O                | 2.0000e+00 |  mol |
| Species Amount:     |            |      |
| :: CH4              | 1.0000e+00 |  mol |
| :: C2H6             | 5.0000e-02 |  mol |
| :: O2               | 1.0000e-16 |  mol |
| :: CO               | 1.0000e-16 |  mol |
| :: H2O              | 2.0000e+00 |  mol |
| :: H2               | 1.0000e-16 |  mol |
| :: CO2              | 1.0000e-16 |  mol |
| :: CH3OH :: CH4O    | 1.0000e-16 |  mol |
| :: H2O(l)           | 1.0000e-16 |  mol |
| :: CH3OH(l) :: CH4O | 1.0000e-16 |  mol |
+---------------------+------------+------+
=== FINAL 

Steam reforming

36 bar
H2O/C = 1.5-2

Steam reforming refers to reaction of hydrocarbons with steam in the presence of a
catalyst.

• Inlet temperature: ~ 600°C.
• Outlet temperature: ~ 750°C.
• Pressure: ~ 36 bar.
• Methane conversion: ~ 30%.



In [2]:
state.temperature(750, "celsius")
#state.set("CH4", 2.0, "mol")

solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== FINAL STATE ===
+---------------------+------------+------+
| Property            |      Value | Unit |
+---------------------+------------+------+
| Temperature         |  1023.1500 |    K |
| Pressure            |    36.0000 |  bar |
| Charge:             | 0.0000e+00 |  mol |
| Element Amount:     |            |      |
| :: H                | 8.3000e+00 |  mol |
| :: C                | 1.1000e+00 |  mol |
| :: O                | 2.0000e+00 |  mol |
| Species Amount:     |            |      |
| :: CH4              | 6.8950e-01 |  mol |
| :: C2H6             | 7.2731e-05 |  mol |
| :: O2               | 1.0000e-16 |  mol |
| :: CO               | 1.8238e-01 |  mol |
| :: H2O              | 1.3617e+00 |  mol |
| :: H2               | 1.4091e+00 |  mol |
| :: CO2              | 2.2797e-01 |  mol |
| :: CH3OH :: CH4O    | 7.0013e-07 |  mol |
| :: H2O(l)           | 1.0000e-16 |  mol |
| :: CH3OH(l) :: CH4O | 1.0000e-16 |  mol |
+---------------------+------------+------+


Autothermal reforming

• Autothermal reforming is the reforming of light hydrocarbons in a mixture of steam and oxygen in the presence of a catalyst.

In [3]:
state.temperature(1000, "celsius")
state.pressure(35, "bar")
state.set("O2", 0.5, "mol")

solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== FINAL STATE ===
+---------------------+------------+------+
| Property            |      Value | Unit |
+---------------------+------------+------+
| Temperature         |  1273.1500 |    K |
| Pressure            |    35.0000 |  bar |
| Charge:             | 0.0000e+00 |  mol |
| Element Amount:     |            |      |
| :: H                | 8.3000e+00 |  mol |
| :: C                | 1.1000e+00 |  mol |
| :: O                | 3.0000e+00 |  mol |
| Species Amount:     |            |      |
| :: CH4              | 3.5665e-02 |  mol |
| :: C2H6             | 6.1195e-07 |  mol |
| :: O2               | 1.8353e-16 |  mol |
| :: CO               | 7.5973e-01 |  mol |
| :: H2O              | 1.6311e+00 |  mol |
| :: H2               | 2.4476e+00 |  mol |
| :: CO2              | 3.0460e-01 |  mol |
| :: CH3OH :: CH4O    | 4.0456e-07 |  mol |
| :: H2O(l)           | 1.0000e-16 |  mol |
| :: CH3OH(l) :: CH4O | 1.0000e-16 |  mol |
+---------------------+------------+------+


The ideal gas stoichiometry is the molar ratio M = 2, where M is defined as:
− M = (H2 – CO2
) / (CO +CO2
)

In [4]:
props = ChemicalProps(state)

M = (state.speciesAmount('H2')[0]-state.speciesAmount('CO2')[0])/(state.speciesAmount('CO')[0]+state.speciesAmount('CO2')[0])

print('M ', M)

#print(props)

M  2.013468105146028


# Methanol synthesis

In [9]:
state.temperature(250, "celsius")
state.pressure(80, "bar")
#state.set("H2O", 0.01, "mol")

solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== FINAL STATE ===
+---------------------+------------+------+
| Property            |      Value | Unit |
+---------------------+------------+------+
| Temperature         |   523.1500 |    K |
| Pressure            |    80.0000 |  bar |
| Charge:             | 0.0000e+00 |  mol |
| Element Amount:     |            |      |
| :: H                | 8.3000e+00 |  mol |
| :: C                | 1.1000e+00 |  mol |
| :: O                | 3.0000e+00 |  mol |
| Species Amount:     |            |      |
| :: CH4              | 8.3661e-01 |  mol |
| :: C2H6             | 1.6167e-05 |  mol |
| :: O2               | 1.0000e-16 |  mol |
| :: CO               | 9.9388e-06 |  mol |
| :: H2O              | 1.3105e+00 |  mol |
| :: H2               | 3.4455e-03 |  mol |
| :: CO2              | 2.6335e-01 |  mol |
| :: CH3OH :: CH4O    | 5.3961e-10 |  mol |
| :: H2O(l)           | 1.1628e+00 |  mol |
| :: CH3OH(l) :: CH4O | 1.0000e-16 |  mol |
+---------------------+------------+------+


# ELemental sulfur

In [18]:
from reaktoro import *
import numpy as np
import pandas as pd
import math as math

from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 H2S O2 S8 H2O")
#gases.set(ActivityModelPengRobinson())

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(120, "celsius")
state.pressure(50, "bar")
state.set("CH4", 1.0, "mol")
state.set("H2S", 0.00005, "mol")
state.set("O2", 0.00000005, "mol")

print("=== INITIAL STATE ===")
print(state)

solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== INITIAL STATE ===
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   393.1500 |    K |
| Pressure        |    50.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0001e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 1.0000e-07 |  mol |
| :: S            | 5.0000e-05 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.0000e+00 |  mol |
| :: H2S          | 5.0000e-05 |  mol |
| :: O2           | 5.0000e-08 |  mol |
| :: S8           | 1.0000e-16 |  mol |
| :: H2O          | 1.0000e-16 |  mol |
+-----------------+------------+------+
=== FINAL STATE ===
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   393.1500 |    K |
| Pressure        |    50.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol 